In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')

In [10]:
train=pd.read_csv('train.csv',sep=',')
test=pd.read_csv('test.csv',sep=',')

In [11]:
train.head()

,ID,Age,Workclass,Education,Marital.Status,Occupation,Relationship,Race,Sex,Hours.Per.Week,Native.Country,Income.Group
0,1,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,2,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,3,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,4,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,5,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 12 columns):
ID                496 non-null int64
Age               496 non-null int64
Workclass         466 non-null object
Education         496 non-null object
Marital.Status    496 non-null object
Occupation        466 non-null object
Relationship      496 non-null object
Race              496 non-null object
Sex               496 non-null object
Hours.Per.Week    496 non-null int64
Native.Country    486 non-null object
Income.Group      495 non-null object
dtypes: int64(3), object(9)
memory usage: 46.6+ KB


In [13]:
train.describe()

,ID,Age,Hours.Per.Week
count,496.000000,496.000000,496.000000
mean,248.500000,38.006048,39.635081
std,143.327132,13.365188,11.808313
min,1.000000,17.000000,1.000000
25%,124.750000,28.000000,40.000000
50%,248.500000,37.000000,40.000000
75%,372.250000,46.000000,41.250000
max,496.000000,90.000000,98.000000


In [14]:
categorical=train.dtypes[train.dtypes=='object'].index

In [20]:
numerical=train.dtypes[train.dtypes=='int64'].index

In [21]:
numerical

Index(['ID', 'Age', 'Hours.Per.Week'], dtype='object')

In [15]:
categorical

Index(['Workclass', 'Education', 'Marital.Status', 'Occupation',
       'Relationship', 'Race', 'Sex', 'Native.Country', 'Income.Group'],
      dtype='object')

In [27]:
# def modecat(x):
#     return mode(x.tolist()).mode[0]
# train[categorical].apply(modecat,axis=0)

In [18]:
def nullvalues(x):
    return sum(x.isnull())
print(train.apply(nullvalues,axis=0))

ID                 0
Age                0
Workclass         30
Education          0
Marital.Status     0
Occupation        30
Relationship       0
Race               0
Sex                0
Hours.Per.Week     0
Native.Country    10
Income.Group       1
dtype: int64


In [16]:
train.apply(lambda x:np.sum(x.isnull()))

ID                 0
Age                0
Workclass         30
Education          0
Marital.Status     0
Occupation        30
Relationship       0
Race               0
Sex                0
Hours.Per.Week     0
Native.Country    10
Income.Group       1
dtype: int64

In [17]:
test.apply(lambda x:np.sum(x.isnull()))

ID                 0
Age                0
Workclass         86
Education          0
Marital.Status     0
Occupation        86
Relationship       1
Race               1
Sex                1
Hours.Per.Week     1
Native.Country    21
dtype: int64

In [19]:
##model
from scipy.stats import mode

In [28]:
def modecat(x):
    return mode(x.tolist()).mode[0]
train[categorical].apply(modecat,axis=0)

C:\Users\chand\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Workclass                    Private
Education                    HS-grad
Marital.Status    Married-civ-spouse
Occupation              Craft-repair
Relationship                 Husband
Race                           White
Sex                             Male
Native.Country         United-States
Income.Group                   <=50K
dtype: object

In [20]:


mode(train['Occupation'].tolist()).mode[0]

C:\Users\chand\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


'Craft-repair'

In [22]:
mode(train['Education'].tolist()).mode[0]

C:\Users\chand\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


'HS-grad'

In [71]:
train['Education'].value_counts()/train.shape[0]

HS-grad         0.314516
Some-college    0.227823
Bachelors       0.159274
Masters         0.060484
Assoc-acdm      0.050403
11th            0.046371
Assoc-voc       0.040323
10th            0.018145
9th             0.018145
7th-8th         0.016129
Doctorate       0.012097
1st-4th         0.010081
5th-6th         0.010081
Prof-school     0.010081
12th            0.004032
Preschool       0.002016
Name: Education, dtype: float64

496

In [61]:
mode(test['Education']).mode[0]

c:\program files\python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


'HS-grad'

In [78]:
mode(train['Occupation']).mode[0]

c:\program files\python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


'Craft-repair'

In [111]:
train['Occupation']=train['Occupation'].fillna("idk")
train['Workclass']=train['Workclass'].fillna('il')
train['Native.Country']=train['Native.Country'].fillna('oo')
train['Income.Group']=train['Income.Group'].fillna('ii')

In [33]:
var_to_impute=['Workclass','Occupation','Native.Country','Income.Group']

for var in var_to_impute:
    train[var].fillna(mode(train[var].tolist()).mode[0],inplace=True)
#     test[var].fillna(mode(train[var]).mode[0],inplace=True)
    

C:\Users\chand\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [34]:
var_to_impute=['Workclass','Occupation','Native.Country']

for var in var_to_impute:
    test[var].fillna(mode(test[var].tolist()).mode[0],inplace=True)

C:\Users\chand\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [35]:
train.apply(lambda x:np.sum(x.isnull()))

ID                0
Age               0
Workclass         0
Education         0
Marital.Status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Hours.Per.Week    0
Native.Country    0
Income.Group      0
dtype: int64

In [38]:
def nullV(x):
    return sum(x.isnull())
train.apply(nullV,axis=0)

ID                0
Age               0
Workclass         0
Education         0
Marital.Status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Hours.Per.Week    0
Native.Country    0
Income.Group      0
dtype: int64